In [21]:
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
import statistics
import numpy as np


In [83]:
tickersymbols = ['AAPL','MSFT','GOOG','IBM','AMZN','FB']

In [90]:
def createDF(tickersymbol): 
    tickerData = yf.Ticker(tickersymbol)
    df = tickerData.history(period='1d', start='2022-01-01', end='2023-01-01')
    df = df.reset_index()
    windowsize = 30
    standarddeviations = 1
    df['Moving average'] = df['Close'].rolling(window=windowsize).mean()
    df = df[['Date', 'Close', 'Moving average']]
    return df
#createDF(tickersymbols[0])
print(createDF(tickersymbols[0]))

                         Date       Close  Moving average
0   2022-01-03 00:00:00-05:00  179.953888             NaN
1   2022-01-04 00:00:00-05:00  177.670013             NaN
2   2022-01-05 00:00:00-05:00  172.943970             NaN
3   2022-01-06 00:00:00-05:00  170.056992             NaN
4   2022-01-07 00:00:00-05:00  170.225052             NaN
..                        ...         ...             ...
246 2022-12-23 00:00:00-05:00  131.127045      142.848545
247 2022-12-27 00:00:00-05:00  129.307251      142.196524
248 2022-12-28 00:00:00-05:00  125.339417      141.459311
249 2022-12-29 00:00:00-05:00  128.889572      140.782096
250 2022-12-30 00:00:00-05:00  129.207794      140.156924

[251 rows x 3 columns]


C:\Users\andre\PycharmProjects\TradingParamTweaker\.venv\Lib\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.



In [64]:
def generatebollingerbands():
    for i in range(len(df['Close']) - windowsize + 1):
        window = df['Close'][i : i + windowsize]
        window_average = sum(window) / windowsize
        std_dev = statistics.stdev(window)
        df.loc[i + windowsize - 1, 'Upper band'] = window_average + standarddeviations*std_dev
        df.loc[i + windowsize - 1, 'Lower band'] = window_average - standarddeviations*std_dev

generatebollingerbands()

In [65]:
fig = px.line(x=df['Date'], y=df['Close'])
fig.add_trace(go.Scatter(x=df['Date'], y=df['Moving average'], mode='lines', name='Moving average'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Upper band'], mode='lines', name='Upper band'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Lower band'], mode='lines', name='Lower band'))
fig.write_html("stock_plot.html")
fig.show()

In [77]:
def transaction(df):
    transaction_history = [] 
    bank = 10000
    startbank = bank
    portfolio = 0
    for index, row in df.iterrows():  
        price = row['Close']
        date = row['Date']
        
        if price <= row['Lower band'] and bank > price: 
            stocks_purchased = 1
            portfolio += stocks_purchased 
            bank -= round(stocks_purchased * price,2)  
            transaction_history.append({'Date': date, ' Signal': 'Buy', 'Stocks in transaction': stocks_purchased,
                                        'Stocks after transaction': portfolio, 'Bank Amount': bank,
                                        'Price': price})
            # transactionhistory-listan tar emot ett transaktionsobjekt innehållande all info vi behöver.

        if price >= row['Upper band'] and portfolio > 0:
            stocks_sold = 1
            portfolio -= stocks_sold
            bank += round(stocks_sold * price, 2)
            transaction_history.append({'Date': date, ' Signal': 'Sell', 'Stocks in transaction': stocks_sold,
                                        'Stocks after transaction': portfolio, 'Bank Amount': bank,
                                        'Price': price})
            # transactionhistory-listan tar emot sälj-händelsen.

        if index == len(df) - 1:
            bank += portfolio * price
            transaction_history.append({'Date': date, ' Signal': 'Sell', 'Stocks in transaction': portfolio,
                                        'Stocks after transaction': portfolio, 'Bank Amount': bank,
                                        'Price': price})
            # Den slutgiltiga försäljningen läggs till i slutet av transaktionshistoriken
            portfolio = 0  # Potfolion töms, då alla aktier är sålda.
    #print(transaction_history)
    return transaction_history[-1]['Bank Amount'] - startbank

transaction(df)



[116.37956176757325, 116.37956176757325, 116.37956176757325, 116.37956176757325, 116.37956176757325, 116.37956176757325, 116.37956176757325, 116.37956176757325, 116.37956176757325, 116.37956176757325]
